In [ ]:
import json
from config import config
from pathlib import Path
import pandas as pd
from collections.abc import Generator
from typing import Any, TypeAlias
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt_tab')
from gensim.utils import tokenize
import en_core_web_sm # python3 -m spacy download en_core_web_sm
spacy_nlp = en_core_web_sm.load()

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('wordnet')

from nltk.corpus import stopwords
from gensim.parsing.preprocessing import remove_stopwords
nltk.download('stopwords', quiet=False)
from nltk.metrics.distance import edit_distance
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('words')

from nltk.corpus import wordnet as wn
from nltk.corpus import words
word_list = set(wn.all_lemmas())

# Download the words corpus

# Access the word list
english_vocab = set(words.words())

In [138]:
RAW_FORMULA1_COMMENTS_FILE = config.RAW_DATA_DIR / 'formula1_comments.ndjson'
RAW_FORMULA1_SUBMISSIONS_FILE = config.RAW_DATA_DIR / 'formula1_submissions.ndjson'
RAW_FORMULA1POINT5_COMMENTS_FILE = config.RAW_DATA_DIR / 'formula1point5_comments.ndjson'
RAW_FORMULA1POINT5_SUBMISSIONS_FILE = config.RAW_DATA_DIR / 'formula1point5_submissions.ndjson'

CommentsDict: TypeAlias = dict[str, Any]
SubmissionsDict: TypeAlias = dict[str, Any]

def load_ndjson(ndjson_file: Path) -> Generator[dict[str, Any]]:
    with open(ndjson_file, 'r', encoding='utf-8') as file:
        for line in file:
            yield json.loads(line)

def load_comments(ndjson_file: Path) -> Generator[CommentsDict]:
    yield from load_ndjson(ndjson_file)

def load_submissions(ndjson_file: Path) -> Generator[SubmissionsDict]:
    yield from load_ndjson(ndjson_file)

In [ ]:
display(tuple(load_submissions(RAW_FORMULA1POINT5_SUBMISSIONS_FILE)))

In [140]:
# formula1_comments = tuple(load_comments(RAW_FORMULA1_COMMENTS_FILE))
# formula1_submissions = tuple(load_submissions(RAW_FORMULA1_SUBMISSIONS_FILE))
formula1point5_comments = tuple(load_comments(RAW_FORMULA1POINT5_COMMENTS_FILE))
formula1point5_submissions = tuple(load_submissions(RAW_FORMULA1POINT5_SUBMISSIONS_FILE))

In [ ]:
# LOADED_COMMENTS_COLUMNS = (
#     'all_awardings',
#     'associated_award',
#     'author',
# )

df = pd.DataFrame(load_comments(RAW_FORMULA1POINT5_COMMENTS_FILE))
display(df.head())
df = df.convert_dtypes()
df.dtypes
display(df.head())

In [ ]:
random_comment = formula1point5_comments[15]['body']
print(random_comment)

In [ ]:
def add_newlines(s):
    return '\n'.join(s[:])

import re
print('|'.join(word_tokenize(random_comment)))
print(re.sub(r'\s+', '|', random_comment))

In [ ]:
'|'.join(tokenize(random_comment))

In [ ]:
print('|'.join(random_comment.split()))
print('|'.join(token.text for token in spacy_nlp(random_comment)))

In [174]:
F1_names= {
    'max verstappen',
    'charles leclerc',
    'sergio perez',
    'george russell',
    'carlos sainz',
    'lewis hamilton',
    'lando norris',
    'esteban ocon',
    'fernando alonso',
    'valtteri bottas',
    'daniel ricciardo',
    'sebastian vettel',
    'kevin magnussen',
    'pierre gasly',
    'lance stroll',
    'mick schumacher',
    'yuki tsunoda',
    'zhou guanyu',
    'alexander albon',
    'nicholas latifi',
    'nyck de vries',
    'nico hulkenberg',
    'oscar piastri',
    'liam lawson',
    'logan sargeant'
}

F1_DRIVERS = {
    'max', 'verstappen',
    'charles', 'leclerc',
    'sergio', 'perez',
    'george', 'russell',
    'carlos', 'sainz',
    'lewis', 'hamilton',
    'lando', 'norris',
    'esteban', 'ocon',
    'fernando', 'alonso',
    'valtteri', 'bottas',
    'daniel', 'ricciardo',
    'sebastian', 'vettel',
    'kevin', 'magnussen',
    'pierre', 'gasly',
    'lance', 'stroll',
    'mick', 'schumacher',
    'yuki', 'tsunoda',
    'zhou', 'guanyu',
    'alexander', 'albon',
    'nicholas', 'latifi',
    'nyck', 'vries',
    'nico', 'hulkenberg',
    'oscar', 'piastri',
    'liam', 'lawson',
    'logan', 'sargeant'
}

Drivers_dict = {
    'max': 'max verstappen',
    'charles': 'charles leclerc',
    'sergio': 'sergio perez',
    'george': 'george russell',
    'carlos': 'carlos sainz',
    'lewis': 'lewis hamilton',
    'lando': 'lando norris',
    'esteban': 'esteban ocon',
    'fernando': 'fernando alonso',
    'valtteri': 'valtteri bottas',
    'daniel': 'daniel ricciardo',
    'sebastian': 'sebastian vettel',
    'kevin': 'kevin magnussen',
    'pierre': 'pierre gasly',
    'lance': 'lance stroll',
    'mick': 'mick schumacher',
    'yuki': 'yuki tsunoda',
    'zhou': 'zhou guanyu',
    'alexander': 'alexander albon',
    'nicholas': 'nicholas latifi',
    'nyck': 'nyck de vries',
    'nico': 'nico hulkenberg',
    'oscar': 'oscar piastri',
    'liam': 'liam lawson',
    'logan': 'logan sargeant',
    
    'verstappen': 'max verstappen',
    'leclerc': 'charles leclerc',
    'perez': 'sergio perez',
    'russell': 'george russell',
    'sainz': 'carlos sainz',
    'hamilton': 'lewis hamilton',
    'norris': 'lando norris',
    'ocon': 'esteban ocon',
    'alonso': 'fernando alonso',
    'bottas': 'valtteri bottas',
    'ricciardo': 'daniel ricciardo',
    'vettel': 'sebastian vettel',
    'magnussen': 'kevin magnussen',
    'gasly': 'pierre gasly',
    'stroll': 'lance stroll',
    'schumacher': 'mick schumacher',
    'tsunoda': 'yuki tsunoda',
    'guanyu': 'zhou guanyu',
    'albon': 'alexander albon',
    'latifi': 'nicholas latifi',
    'vries': 'nyck de vries',
    'hulkenberg': 'nico hulkenberg',
    'piastri': 'oscar piastri',
    'lawson': 'liam lawson',
    'sargean': 'logan sargeant',
}

F1_DRIVERS = {
    'max', 'verstappen',
    'charles', 'leclerc',
    'sergio', 'perez',
    'george', 'russell',
    'carlos', 'sainz',
    'lewis', 'hamilton',
    'lando', 'norris',
    'esteban', 'ocon',
    'fernando', 'alonso',
    'valtteri', 'bottas',
    'daniel', 'ricciardo',
    'sebastian', 'vettel',
    'kevin', 'magnussen',
    'pierre', 'gasly',
    'lance', 'stroll',
    'mick', 'schumacher',
    'yuki', 'tsunoda',
    'zhou', 'guanyu',
    'alexander', 'albon',
    'nicholas', 'latifi',
    'nyck', 'vries',
    'nico', 'hulkenberg',
    'oscar', 'piastri',
    'liam', 'lawson',
    'logan', 'sargeant'
}

F1_VOCABULARY = F1_DRIVERS

In [176]:
def normalize(comment):
    comment = comment.lower()
    comment = re.sub(r'http\S+', '', comment)
    comment = re.sub(r'[^a-z\s\d]', '', comment)
    return comment


def tokenize(text):
    tokens = []
    for token in spacy_nlp(text):
        tokens.append(token.text)
    return tokens


def remove_stopword(tokens, stop_words=None):
    if stop_words is None:
        stop_words = set(stopwords.words('english'))
    new_tokens = []
    for word in tokens:
        if word not in stop_words:
            new_tokens.append(word)
    return new_tokens


def lemmatize(tokens):

    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]

    return lemmatized_tokens


def NER(tokens):
    s = ' '.join(tokens)

    doc = spacy_nlp(s)

    entities = [(ent.text, ent.label_) for ent in doc.ents]
    return entities

def correct_spelling(word):
    if word in F1_VOCABULARY:
        return word
    
    min_distance = float('inf')
    corrected_word = word
    for term in F1_VOCABULARY:
        distance = edit_distance(word, term)
        if distance < min_distance and distance <= max(1, len(word)//3):  # Allow a maximum edit distance of 33%
            min_distance = distance
            corrected_word = term
    # print(word, corrected_word, min_distance)
    return corrected_word

def combine_names(tokens):
    combined_tokens = []
    skip_next = False

    for i, word in enumerate(tokens):
        if skip_next:
            skip_next = False
            continue

        # Check if the current word and the next word form a driver name
        if i + 1 < len(tokens):
            combined_word = f'{word} {tokens[i + 1]}'
            if combined_word in F1_names:
                combined_tokens.append(combined_word)
                skip_next = True
                continue

        # Check if the current word alone matches a driver name
        if word in F1_DRIVERS:
            combined_tokens.append(Drivers_dict[word])
        else:
            combined_tokens.append(word)

    return combined_tokens

In [ ]:
quote = 'i am really not a big fan of msx verstappening i am much more a fan of charles lecllerf fuck maxipad verstappen'

In [147]:
# method to perform PoS tagging using NLTK
# INPUT: Raw text
# Output: A list of tuples including each word and its respective PoS tag
def pos_tagging_nltk(text):
    
    # first we tokenize the raw text
    tokenized_text = nltk.word_tokenize(text)
    
    # extract the PoS tags
    pos_tags = nltk.pos_tag(tokenized_text)
    
    # return the PoS tags
    return pos_tags


In [ ]:
# for comment in ()r comment i
comment = random_comment
comment = 'I am really not a big fan of msx verstappening, i am much more a fan of charles lecllerf'


normalized = normalize(comment)
tokens = tokenize(normalized)

corrected = []
for word in tokens:
    corrected.append(correct_spelling(word))

combined = combine_names(corrected)
lemmatized = lemmatize(combined)
no_stopwords = remove_stopword(lemmatized)
named_entities = NER(comment)

print(comment)
print(combined)
print(lemmatized)
print(no_stopwords)
print(named_entities)
# print(pos_tagging_nltk(' '.join(lemmatized)))

In [ ]:
levenshtein_distance('max verstappen', 'max verstappening')
word = correct_spelling("max verstappening")
print(word)